In [168]:
# Import selenium and its dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# import pandas 
import pandas as pd

In [61]:
# create a dataframe to hold the scrapped data
df = pd.DataFrame()

# launch the browser driver
driver = webdriver.Firefox()

# create a list of alphabets to loop through
alphabet = list("abcdefghijklmnopqrstuvwxyz")


In [1]:
# loop through each letter
for letter in alphabet:
    
    print(f"Now Scraping: {letter}")
    
    # go to the main CPSO website with the last name search box
    driver.get("https://doctors.cpso.on.ca/?doctors")
    
    # clear the searchbox, enter the letter as the last name and then hit enter to searcn
    driver.find_element(By.ID, "p_lt_ctl01_pageplaceholder_p_lt_ctl02_CPSO_AllDoctorsSearch_txtLastNameQuick").clear()
    driver.find_element(By.ID, "p_lt_ctl01_pageplacehoslder_p_lt_ctl02_CPSO_AllDoctorsSearch_txtLastNameQuick").send_keys(letter)
    driver.find_element(By.ID, "p_lt_ctl01_pageplaceholder_p_lt_ctl02_CPSO_AllDoctorsSearch_txtLastNameQuick").send_keys(Keys.ENTER)
    
    # wait for the last page number to load and then get it
    # this also helps the browser wait before getting any other elements
    # Other wise break out of the loop and crash 
    try:
        last_page_num = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "p_lt_ctl01_pageplaceholder_p_lt_ctl03_CPSO_DoctorSearchResults_lnbLastPage"))
            )
    except:
        break
    
    # get the last page number as an int
    last_page_num = driver.find_element(By.ID, "p_lt_ctl01_pageplaceholder_p_lt_ctl03_CPSO_DoctorSearchResults_lnbLastPage").text
    last_page_num = int(last_page_num)
    
    # loop through the nav bar as a multiple of last page number / 5
    for i in range(round(last_page_num/5)):
        
        # loop through the 5 pages that are in the nav bar
        for i in range(0, 5):
            
            # loop through each article tag on the page that contains the doctor information
            for article in driver.find_elements(By.TAG_NAME, "article"):
                
                # get the doctor's name
                doctor_name = article.find_element(By.TAG_NAME, "a").text
                
                # get the address
                address = article.find_element(By.TAG_NAME, "p").text
                
                # save it to the data set
                data = {"doctor name": doctor_name, "address": address}
                
                # append it to the df
                df = pd.concat([df, pd.DataFrame(data, index=[0])], ignore_index=True)
            
            # click the next page if it exists other wise click the next group button
            try:
                driver.find_element(By.ID, f"p_lt_ctl01_pageplaceholder_p_lt_ctl03_CPSO_DoctorSearchResults_rptPages_ctl0{i}_lnbPage").click()
            except:
                pass
            
        # if the last page number exists then click the next page    
        # if it doesnt exist then just go to the next letter as the driver is already on the last page
        driver.find_element(By.ID, "p_lt_ctl01_pageplaceholder_p_lt_ctl03_CPSO_DoctorSearchResults_lnbNextGroup").click()

NameError: name 'alphabet' is not defined

In [ ]:
# save the df
df.to_csv("./csv/doctors.csv")

In [222]:
# drop any address that is null 
df = df[~df["address"].isna()]

# split the address based on the "\n"
df['address'] = df["address"].str.split("\n")

# use this regex patter to find the string that has the postal code
# that string will have the city
pattern = r"\b[a-zA-Z]\d[a-zA-Z] \d[a-zA-Z]\d\b"

# loop through each row in the df
for iter, row in df.iterrows():
    
    # go through each string and find the postal code
    for string in row["address"]:
        postal_code = re.search(pattern, string)
        
        # if there is a postal code than process the string to find the city
        if postal_code:
            string = string.replace(postal_code.group(), "")
            string = string.replace("ON", "").strip().title()
            row['city'] = string
            


In [252]:
# save the doctor df
df.to_csv("./csv/doctors.csv")

# save the cumulative city count
df['city'].value_counts().to_csv('./csv/doctor_city_count.csv')